# Implementação algoritmo REINFORCEMENT no RTB for python com Swift e manipulador panda.


In [50]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
from IPython.display import clear_output
from scipy.signal import convolve, gaussian
from IPython.display import HTML

import data_panda as rbt
device = 'cpu'
%matplotlib inline


#j3 range -0.08 a 3.75  #j2 range -0.07 a -3. #j1 range -1.8 a 1.76

#j3 range -0.08 a 3.75  #j2 range -0.07 a -3. #j1 range -1.8 a 1.76

### Sistema observável e com medidas dos ângulos disponíveis


    Observation:
        Type: Box(4)
        Num     Observation               Min                     Max

        0       Joint1                   -4.8                    4.8
        1       Joint2                    -Inf                    Inf
        2       Joint3                -0.418 rad (-24 deg)    0.418 rad (24 deg)
        
    Actions:
        Type: Discrete 3^3 27 actions
        
        Num   Three actions for each joint
         0     do n joint j
         1     increment angle of joint j
        -1     decrement angle of joint j

        #j3 range 0.0 a 3.7
        #j2 range 0.0 a -3.
        #j1 range -1.7 a 1.7



        



In [51]:
state_shape = 3
env=rbt.Panda_RL()
config = rbt.load_config("config_REINF.yaml")
agent=rbt.REINF(state_shape, device, epsilon=1).to(device)
env.renderize=True #stop robot viewing
env.ceil=config['ceil']

In [52]:
# env.ceil=False

In [53]:
env.panda

ERobot: panda (by Franka Emika), 7 joints (RRRRRRR), 1 gripper, geometry, collision
┌─────┬──────────────┬───────┬─────────────┬────────────────────────────────────────────────┐
│link │     link     │ joint │   parent    │              ETS: parent to link               │
├─────┼──────────────┼───────┼─────────────┼────────────────────────────────────────────────┤
│   0 │ panda_link0  │       │ BASE        │                                                │
│   1 │ panda_link1  │     0 │ panda_link0 │ SE3(0, 0, 0.333) ⊕ Rz(q0)                      │
│   2 │ panda_link2  │     1 │ panda_link1 │ SE3(-90°, -0°, 0°) ⊕ Rz(q1)                    │
│   3 │ panda_link3  │     2 │ panda_link2 │ SE3(0, -0.316, 0; 90°, -0°, 0°) ⊕ Rz(q2)       │
│   4 │ panda_link4  │     3 │ panda_link3 │ SE3(0.0825, 0, 0; 90°, -0°, 0°) ⊕ Rz(q3)       │
│   5 │ panda_link5  │     4 │ panda_link4 │ SE3(-0.0825, 0.384, 0; -90°, -0°, 0°) ⊕ Rz(q4) │
│   6 │ panda_link6  │     5 │ panda_link5 │ SE3(90°, -0°, 0°) ⊕ Rz(q5

### Main loop



Intializing with a given state

In [54]:
env.start_scene()

In [55]:
# env.reset_j1=[-1.7,1.7]
# env.reset_j1=[-1.7,-1.]

# env.reset_j1= [-0.3,0.3] #[-1.7,-1.] 
# env.reset_j2= [-1,0.06]
# env.reset_j3= [0,3.75]
env.reset()

array([-1.6 , -2.6 ,  0.71])

In [56]:
env.panda.q=env.q_goal
env.sig_R=1
print(env.step([1,1,-1]))
print(env.fitness())
# print(env.step([0,0,1]))
# print(env.fitness())

(array([ 0.36, -0.83,  3.68]), 10.684961420527879, False, ['Running', ''])
(2.9809296978283035e-05, 0.010000000000023646)


/home/taniel/mambaforge/envs/RTP/lib/python3.10/site-packages/roboticstoolbox/robot/Link.py:1041: FutureWarning: base kwarg is deprecated, use pose instead
  warn("base kwarg is deprecated, use pose instead", FutureWarning)


In [57]:
print(env.step([0,0,-1]))
print(env.fitness())

(array([ 0.36, -0.83,  3.67]), -15.052227904166774, False, ['Running', ''])
(5.940605089789969e-05, 0.020000000000012206)


In [58]:
print(env.step([1,0,0]))
print(env.fitness())

(array([ 0.37, -0.83,  3.67]), -15.135861042651866, False, ['Running', ''])
(0.00027687448776614023, 0.029999999999999107)


In [104]:
NAME_DIR="reinf_6"
n_steps=500
agent.load_weights(NAME_DIR,model="best")
states, actions, rewards, info=agent.generate_trajectory(env, n_steps=n_steps)
print(f'Final score:{np.sum(rewards)} in {len(rewards)} steps')
print(f"Status: {info[0]} {info[1]}")

Final score:1147.893241243554 in 306 steps
Status: Termination Collided


In [98]:
agent.load_weights(NAME_DIR,model="last")
states, actions, rewards, info=agent.generate_trajectory(env, n_steps=n_steps)
print(f'Final score:{np.sum(rewards)} in {len(rewards)} steps')
print(f"Status: {info[0]} {info[1]}")

Final score:951.6296577837372 in 500 steps
Status: Running 


In [14]:
#agent.generate_trajectory(env,n_steps=config['tmax'])
agent.generate_trajectory(env,n_steps=300)

([array([-1.6 , -2.6 ,  0.71]),
  array([-1.59, -2.6 ,  0.72]),
  array([-1.58, -2.6 ,  0.73]),
  array([-1.58, -2.59,  0.74]),
  array([-1.57, -2.59,  0.75]),
  array([-1.56, -2.59,  0.76]),
  array([-1.55, -2.59,  0.77]),
  array([-1.55, -2.58,  0.78]),
  array([-1.54, -2.58,  0.79]),
  array([-1.53, -2.58,  0.8 ]),
  array([-1.52, -2.58,  0.81]),
  array([-1.51, -2.58,  0.82]),
  array([-1.5 , -2.58,  0.83]),
  array([-1.5 , -2.57,  0.84]),
  array([-1.49, -2.57,  0.85]),
  array([-1.49, -2.56,  0.86]),
  array([-1.49, -2.55,  0.87]),
  array([-1.48, -2.55,  0.88]),
  array([-1.47, -2.55,  0.89]),
  array([-1.47, -2.54,  0.9 ]),
  array([-1.46, -2.54,  0.91]),
  array([-1.46, -2.53,  0.92]),
  array([-1.45, -2.53,  0.93]),
  array([-1.44, -2.53,  0.94]),
  array([-1.44, -2.52,  0.95]),
  array([-1.44, -2.51,  0.96]),
  array([-1.43, -2.51,  0.97]),
  array([-1.42, -2.51,  0.98]),
  array([-1.42, -2.5 ,  0.99]),
  array([-1.41, -2.5 ,  1.  ]),
  array([-1.4 , -2.5 ,  1.01]),
  array(

**Let us record a video of trained agent**

In [ ]:
# # Animate learned policy
# save_dir='./videos/'
# #env = make_env(env_name)
# generate_animation(env, agent, save_dir=save_dir)
# [filepath] = glob.glob(os.path.join(save_dir, '*.mp4'))

# display_animation(filepath)

In [ ]:
#TODO

# Run this on another environment in OpenAI Gym
# Create a robotic environment with more actions
#